In [ ]:
import requests
from openai.embeddings_utils import get_embedding
import openai
import os
import json
from pprint import pprint
# dotenv
from dotenv import load_dotenv
os.environ.pop("OPENAI_API_KEY")
os.environ.pop("OPENAI_ENDPOINT")
os.environ.pop("AI_SEARCH_SERVICE")
os.environ.pop("AI_SEARCH_KEY")
load_dotenv()

In [ ]:
openai.api_type = "azure"
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_ENDPOINT']
openai.api_version = "2022-12-01"


In [ ]:


# send a query to the AI Search service
service_name = os.environ['AI_SEARCH_SERVICE']
index_name = 'faqdemo'
api_version = '2023-11-01'
api_key = os.environ['AI_SEARCH_KEY']
endpoint = f'https://{service_name}.search.windows.net/indexes/{index_name}/docs/search?api-version={api_version}'


In [ ]:
question = "Can I skip the 30 days and just start with pay-as-you-go pricing?"

In [52]:
def build_payload_vector_search(question, k=7):
    return json.dumps({
        "count": True,
        "select": "id, question, answer, Tags",
        "vectorQueries": [
            # {
            #     "vector": get_embedding(question, engine="text-embedding-ada-002"),
            #     "k": 7,
            #     "fields": "AnswerVector",
            #     "kind": "vector",
            #     "exhaustive": True
            # },
            {
                "vector": get_embedding(question, engine="text-embedding-ada-002"),
                "k": k,
                "fields": "QuestionVector",
                "kind": "vector",
                "exhaustive": True
            }
        ]
    })

In [55]:
def build_payload_hybrid_search(question, k=7, top=7):
    return json.dumps({
        "count": True,
        "search": question,
        "select": "id, question, answer, Tags",
        "top": top,
        "vectorQueries": [
            {
                "vector": get_embedding(question, engine="text-embedding-ada-002"),
                "k": k,
                "fields": "AnswerVector",
                "kind": "vector",
                "exhaustive": True
            }
        ]
    })

In [63]:
def build_payload_search(question, top=7):
    return json.dumps({
        "count": True,
        "search": question,
        "select": "id, question, answer, Tags",
        "top": top
    })

In [65]:
payload = build_payload_search(question, 3)
headers = {
  'Content-Type': 'application/json',
  'api-key': api_key
}
response = requests.request("POST", endpoint, headers=headers, data=payload)
print(f"question: {question}")
# pprint(response.json())
print("response:")
for r in response.json()["value"]:
    print(r["question"])
    print(r["answer"])
    print(r["Tags"])
    print(r["@search.score"])
    print(r["id"])
    print()

question: What are the benefits of using Azure Kubernetes Service (AKS) for container orchestration compared to managing my own Kubernetes clusters?
Are security updates applied to AKS agent nodes?
AKS patches CVEs that have a 'vendor fix' every week. CVEs without a fix are waiting on a 'vendor fix' before it can be remediated. The AKS images are automatically updated inside of 30 days. We recommend you apply an updated Node Image on a regular cadence to ensure that latest patched images and OS patches are all applied and current. You can do this using one of the following methods:\n\nManually, through the Azure portal or the Azure CLI.\nBy upgrading your AKS cluster. The cluster upgrades cordon and drain nodes automatically and then bring a new node online with the latest Ubuntu image and a new patch version or a minor Kubernetes version. For more information, see Upgrade an AKS cluster.\nBy using node image upgrade.
['FAQ', 'Azure']
8.928544
9e4918e7-b288-447a-a1ef-c622b331ed72

Can 

In [60]:
payload = build_payload_vector_search(question, 1)
headers = {
  'Content-Type': 'application/json',
  'api-key': api_key
}
response = requests.request("POST", endpoint, headers=headers, data=payload)
print(f"question: {question}")
print("response:")
for r in response.json()["value"]:
    print(r["question"])
    print(r["answer"])
    print(r["Tags"])
    print(r["@search.score"])
    print(r["id"])
    print()



question: What are the benefits of using Azure Kubernetes Service (AKS) for container orchestration compared to managing my own Kubernetes clusters?
Can I move my AKS clusters from the current Azure subscription to another?
Moving your AKS cluster and its associated resources between Azure subscriptions isn't supported.
['FAQ', 'Azure']
0.8773617
fc546018-3440-4fa8-9e1b-79f5d850f419



In [61]:

payload = build_payload_hybrid_search(question, 1, 3)
headers = {
  'Content-Type': 'application/json',
  'api-key': api_key
}

response = requests.request("POST", endpoint, headers=headers, data=payload)
print(f"qustion: {question}")
print("response:")
for r in response.json()["value"]:
    print(r["question"])
    print(r["answer"])
    print(r["Tags"])
    print(r["@search.score"])
    print(r["id"])
    print()

qustion: What are the benefits of using Azure Kubernetes Service (AKS) for container orchestration compared to managing my own Kubernetes clusters?
What is platform support, and what does it include?
Platform support is a reduced support plan for unsupported 'N-3' version clusters. Platform support only includes Azure infrastructure support. Platform support doesn't include anything related to the following: Kubernetes functionality and components, Cluster or node pool creation, Hotfixes, Bug fixes, Security patches, Retired components. For more information on restrictions, see the platform support policy. AKS relies on the releases and patches from Kubernetes, which is an Open Source project that only supports a sliding window of three minor versions. AKS can only guarantee full support while those versions are being serviced upstream. Since there's no more patches being produced upstream, AKS can either leave those versions unpatched or fork. Due to this limitation, platform support 

In [46]:
def answer_question_vector(question):
    payload = build_payload_vector_search(question)
    headers = {
      'Content-Type': 'application/json',
        'api-key': api_key
    }
    response = requests.request("POST", endpoint, headers=headers, data=payload)
    return response.json()["value"]

def answer_question_hybrid(question):
    payload = build_payload_hybrid_search(question)
    headers = {
      'Content-Type': 'application/json',
        'api-key': api_key
    }
    response = requests.request("POST", endpoint, headers=headers, data=payload)
    return response.json()["value"]

In [48]:
question = "What are the benefits of using Azure Kubernetes Service (AKS) for container orchestration compared to managing my own Kubernetes clusters?"
answer_hybrid = answer_question_hybrid(question)
answer_vector = answer_question_vector(question)

print(f"hybrid={answer_hybrid}")
print(f"vector={answer_vector}")

hybrid={'@search.score': 0.031159421429038048, 'id': '150e4868-2afa-4c8e-92d5-d2d4b5c096d6', 'question': 'What is platform support, and what does it include?', 'answer': "Platform support is a reduced support plan for unsupported 'N-3' version clusters. Platform support only includes Azure infrastructure support. Platform support doesn't include anything related to the following: Kubernetes functionality and components, Cluster or node pool creation, Hotfixes, Bug fixes, Security patches, Retired components. For more information on restrictions, see the platform support policy. AKS relies on the releases and patches from Kubernetes, which is an Open Source project that only supports a sliding window of three minor versions. AKS can only guarantee full support while those versions are being serviced upstream. Since there's no more patches being produced upstream, AKS can either leave those versions unpatched or fork. Due to this limitation, platform support doesn't support anything from